In [1]:
import torch
inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your (x^1)
     [0.55, 0.87, 0.66], # journey (x^2)
     [0.57, 0.85, 0.64], # starts (x^3)
     [0.22, 0.58, 0.33], # with (x^4)
     [0.77, 0.25, 0.10], # one (x^5)
     [0.05, 0.80, 0.55]] # step (x^6)
)

In [2]:
query = inputs[1] # Query vector (x^2)
attn_scores_2 = torch.empty(inputs.shape[0]) # To store attention scores
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query) # Dot product
print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [4]:
res = 0
for idx, element in enumerate(inputs[0]):
    res += inputs[0][idx] * query[idx]

print(res)  # Should match attn_scores_2[0]
print(torch.dot(inputs[0], query))  # Direct dot product for verification  


tensor(0.9544)
tensor(0.9544)


In [5]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()
print("Attention weights:", attn_weights_2_tmp)
print("Sum:", attn_weights_2_tmp.sum())

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


In [6]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_scores_2)
print("Naive Softmax Weights:", attn_weights_2_naive)
print("Sum:", attn_weights_2_naive.sum())

Naive Softmax Weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


In [7]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
print("PyTorch Softmax Weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

PyTorch Softmax Weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


In [10]:
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i] * x_i
print("Context Vector:", context_vec_2)

Context Vector: tensor([0.4419, 0.6515, 0.5683])


In [11]:
attn_scores = torch.empty((inputs.shape[0], inputs.shape[0]))
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

print("Attention Scores Matrix:\n", attn_scores)

Attention Scores Matrix:
 tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [13]:
attn_scores = inputs @ inputs.T
print("Attention Scores Matrix using @ operator:\n", attn_scores)

Attention Scores Matrix using @ operator:
 tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [14]:
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [15]:
print("All row sums:", attn_weights.sum(dim=-1))

All row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [17]:
all_context_vecs = attn_weights @ inputs
print(attn_weights.shape)
print(inputs.shape)
print("All Context Vectors:\n", all_context_vecs)

torch.Size([6, 6])
torch.Size([6, 3])
All Context Vectors:
 tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [33]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2 

# Initialize the three weight matrices
torch.manual_seed(123)  # For reproducibility
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

# We set requires_grad=False for simplicity in this example, if we were training the model, we would set it to True
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value
print("Query Vector:", query_2)

# In the weight matrices W, the term "weight" refers to the learnable parameters that transform the input vectors into query, key, and value vectors.
# These are not the attention weights computed later in the attention mechanism.

# Weight parameters are the fundamental, learned coefficients that define how input data is processed within the model.
# Attention weights, on the other hand, are dynamic coefficients computed during the attention mechanism that determine 
# the relevance of different input elements to each other based on their content. 
# They are derived from the interactions between the query and key vectors and are used to weight the value vectors

keys = inputs @ W_key
values = inputs @ W_value
print("keys.shape:", keys.shape)
print("values.shape:", values.shape)


Query Vector: tensor([0.4306, 1.4551])
keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


In [28]:
keys_2 = keys[1]
attn_score_22 = query_2.dot(keys_2)
print("Attention Score for x^2 with x^2:", attn_score_22)

Attention Score for x^2 with x^2: tensor(1.8524)


In [29]:
attn_scores_2 = query_2 @ keys.T
print("Attention Scores for x^2 with all keys:", attn_scores_2)

Attention Scores for x^2 with all keys: tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


In [30]:
# To go from attention scores to attention weights, we apply the softmax function
d_k = keys.shape[-1] # Dimension of the key vectors
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=0) # Scale by sqrt(d_k)
print("Scaled Attention Weights:", attn_weights_2)

# The reason for normalization by the embedding dimension's square root (sqrt(d_k)) is to prevent the dot products from becoming too large in magnitude.
# When the dimensionality of the key and query vectors is high, the dot products can grow
# very large, which can push the softmax function into regions where it has extremely small gradients.
# This can make the model harder to train effectively. By scaling the dot products down by sqrt(d_k),
# we help keep the values in a range that is more suitable for the softmax function, ensuring better gradient flow during training.
# This technique was introduced in the "Attention is All You Need" paper by Vaswani et al.
# This is also called scaled dot-product attention.

Scaled Attention Weights: tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


In [31]:
# The final step is to compute the context vector as a weighted sum of the value vectors
context_vec_2 = attn_weights_2 @ values
print("Context Vector:", context_vec_2)

# This is the context vector z^2 for the input x^2, computed using the attention mechanism with learned query, key, and value projections.

Context Vector: tensor([0.3061, 0.8210])


In [ ]:
# Why query, key and value?
# The concepts of query, key, and value in the attention mechanism are inspired by information retrieval systems.
# In such systems, a "query" is used to search for relevant "keys" in a database, and the associated "values" are the information retrieved based on that search.
# In the context of neural networks and attention mechanisms, this analogy helps to conceptualize how different parts of the input data interact with each other.
# The query vector represents the current focus of attention (what we are looking for),  
# the key vectors represent the features of the input data that can be matched against the query (what we have),
# and the value vectors represent the actual information to be aggregated based on the attention scores (what we retrieve).

In [36]:
# Self attention class
import torch.nn as nn
class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))
    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value
        attn_scores = queries @keys.T # omega shape: (seq_len, seq_len)
        attn_weights = torch.softmax(attn_scores / (keys.shape[-1]**0.5), dim=-1)
        context_vec = attn_weights @ values
        return context_vec
    
torch.manual_seed(123)
ea_v1 = SelfAttention_v1(d_in, d_out)
print(ea_v1(inputs))    


tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [37]:
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
    
    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        attn_scores = queries @ keys.T # omega shape: (seq_len, seq_len)
        attn_weights = torch.softmax(attn_scores / (keys.shape[-1]**0.5), dim=-1)
        context_vec = attn_weights @ values
        return context_vec

In [47]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))


tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


In [48]:
# nn.Linear in SelfAttention_v2 uses a different weight initialization strategy compared to manually initializing weights as done in SelfAttention_v1.
# nn.Linear typically uses Kaiming Uniform or Xavier Uniform initialization methods, which are designed to maintain the variance of activations through layers, promoting better convergence during training.
# To prove that these produce similar results, we can manually set the weights of SelfAttention_v2 to match those of SelfAttention_v1 after initialization.
# Need to transpose the weights because of the way nn.Linear stores weights
# in transposed form
sa_v2.W_query.weight.data = ea_v1.W_query.data.T.clone()
sa_v2.W_key.weight.data = ea_v1.W_key.data.T.clone()
sa_v2.W_value.weight.data = ea_v1.W_value.data.T.clone()
print(sa_v2(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [49]:
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)
attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores/ keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[0.1551, 0.2104, 0.2059, 0.1413, 0.1074, 0.1799],
        [0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820],
        [0.1503, 0.2256, 0.2192, 0.1315, 0.0914, 0.1819],
        [0.1591, 0.1994, 0.1962, 0.1477, 0.1206, 0.1769],
        [0.1610, 0.1949, 0.1923, 0.1501, 0.1265, 0.1752],
        [0.1557, 0.2092, 0.2048, 0.1419, 0.1089, 0.1794]],
       grad_fn=<SoftmaxBackward0>)


In [50]:
context_length = attn_scores.shape[0] # Number of tokens in the input sequence
mask_simple = torch.tril(torch.ones((context_length, context_length)))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [51]:
masked_simple = attn_weights * mask_simple
print("Masked Attention Weights:\n", masked_simple)

Masked Attention Weights:
 tensor([[0.1551, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1500, 0.2264, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1503, 0.2256, 0.2192, 0.0000, 0.0000, 0.0000],
        [0.1591, 0.1994, 0.1962, 0.1477, 0.0000, 0.0000],
        [0.1610, 0.1949, 0.1923, 0.1501, 0.1265, 0.0000],
        [0.1557, 0.2092, 0.2048, 0.1419, 0.1089, 0.1794]],
       grad_fn=<MulBackward0>)


In [52]:
# Now renormalize the masked weights so that they sum to 1
row_sums = masked_simple.sum(dim=-1, keepdim=True)
print(row_sums)
masked_simple_norm = masked_simple / row_sums
print("Renormalized Masked Attention Weights:\n", masked_simple_norm)

tensor([[0.1551],
        [0.3764],
        [0.5952],
        [0.7024],
        [0.8248],
        [1.0000]], grad_fn=<SumBackward1>)
Renormalized Masked Attention Weights:
 tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3986, 0.6014, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2526, 0.3791, 0.3683, 0.0000, 0.0000, 0.0000],
        [0.2265, 0.2839, 0.2794, 0.2103, 0.0000, 0.0000],
        [0.1952, 0.2363, 0.2331, 0.1820, 0.1534, 0.0000],
        [0.1557, 0.2092, 0.2048, 0.1419, 0.1089, 0.1794]],
       grad_fn=<DivBackward0>)


In [57]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask == 1, -torch.inf)
print("Masked Attention Scores:\n", masked)
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=-1)
print("Masked Attention Weights with -inf:\n", attn_weights)

Masked Attention Scores:
 tensor([[0.9231,   -inf,   -inf,   -inf,   -inf,   -inf],
        [1.2705, 1.8524,   -inf,   -inf,   -inf,   -inf],
        [1.2544, 1.8284, 1.7877,   -inf,   -inf,   -inf],
        [0.6973, 1.0167, 0.9941, 0.5925,   -inf,   -inf],
        [0.6114, 0.8819, 0.8626, 0.5121, 0.2707,   -inf],
        [0.8995, 1.3165, 1.2871, 0.7682, 0.3937, 1.0996]],
       grad_fn=<MaskedFillBackward0>)
Masked Attention Weights with -inf:
 tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3986, 0.6014, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2526, 0.3791, 0.3683, 0.0000, 0.0000, 0.0000],
        [0.2265, 0.2839, 0.2794, 0.2103, 0.0000, 0.0000],
        [0.1952, 0.2363, 0.2331, 0.1820, 0.1534, 0.0000],
        [0.1557, 0.2092, 0.2048, 0.1419, 0.1089, 0.1794]],
       grad_fn=<SoftmaxBackward0>)


In [58]:
# Add dropout to prevent overfitting
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5)
example = torch.ones(6, 6)
print(dropout(example))
print(dropout(attn_weights))

tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])
tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.7582, 0.7366, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.5587, 0.4206, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.3068, 0.0000],
        [0.3115, 0.4183, 0.0000, 0.2839, 0.2178, 0.3588]],
       grad_fn=<MulBackward0>)


In [59]:
batch = torch.stack((inputs, inputs), dim=0)  # Shape: (2, 6, 3)
print(batch.shape)

torch.Size([2, 6, 3])


In [64]:
class CausalAttention(nn.Module):
    """
    Causal (Masked) Self-Attention module for decoder-only models like GPT.
    Prevents each position from attending to future positions in the sequence.
    """
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
        """
        Args:
            d_in: Input embedding dimension
            d_out: Output dimension (attention head dimension)
            context_length: Maximum sequence length (for creating causal mask)
            dropout: Dropout probability for attention weights
            qkv_bias: Whether to include bias in Q, K, V projections
        """
        super().__init__()
        # Learnable linear projections for Query, Key, Value
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        
        # Create a causal mask (upper triangular matrix of 1s)
        # register_buffer: not a trainable parameter, but moves with model (e.g., to GPU)
        # torch.triu creates upper triangle: 1s above diagonal, 0s on/below diagonal
        # This mask blocks attention to future positions
        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        """
        Args:
            x: Input tensor of shape (batch_size, num_tokens, d_in)
        
        Returns:
            context_vec: Output tensor of shape (batch_size, num_tokens, d_out)
        """
        b, num_tokens, d_in = x.shape
        
        # Project input into Query, Key, Value spaces
        keys = self.W_key(x)       # Shape: (batch, num_tokens, d_out)
        queries = self.W_query(x)  # Shape: (batch, num_tokens, d_out)
        values = self.W_value(x)   # Shape: (batch, num_tokens, d_out)

        # Compute attention scores: Q @ K^T
        # transpose(1, 2) swaps token and d_out dimensions, keeping batch dimension first
        # Result: how much each token (row) should attend to every other token (column)
        attn_scores = queries @ keys.transpose(1, 2)  # Shape: (batch, num_tokens, num_tokens)
        
        # Apply causal mask: set future positions to -inf
        # [:num_tokens, :num_tokens] allows variable sequence lengths
        # After softmax, -inf becomes 0 (no attention to future)
        attn_scores.masked_fill_(self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
        
        # Scaled dot-product attention: divide by sqrt(d_k) to prevent large values
        # Then apply softmax to get probabilities that sum to 1 per row
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        
        # Apply dropout to attention weights (randomly zero some connections during training)
        attn_weights = self.dropout(attn_weights)
        
        # Compute context vectors as weighted sum of values
        # Each output token is a combination of value vectors weighted by attention
        context_vec = attn_weights @ values  # Shape: (batch, num_tokens, d_out)
        return context_vec


In [63]:
# Test the CausalAttention module
torch.manual_seed(123)
context_length = batch.shape[1]  # Get sequence length from batch
ca = CausalAttention(d_in, d_out, context_length, dropout=0.0)
context_vecs = ca(batch)
print("context_vecs.shape:", context_vecs.shape)

context_vecs.shape: torch.Size([2, 6, 2])


In [65]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(
                d_in, d_out, context_length, dropout, qkv_bias                
            )
            for _ in range(num_heads)]
        )
    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)

In [66]:
torch.manual_seed(123)
context_length = batch.shape[1]  # Get sequence length from batch
mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, dropout=0.0, num_heads=2)
context_vecs = mha(batch)
print("context_vecs.shape:", context_vecs.shape)
print("Context Vectors from Multi-Head Attention:\n", context_vecs)


context_vecs.shape: torch.Size([2, 6, 4])
Context Vectors from Multi-Head Attention:
 tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
